In [1]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)
soundstream.load("content/soundstream/results3/soundstream.9000.pt")
soundstream.to("cuda")



2023-01-10 14:13:44 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-10 14:13:44.277071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 14:13:45.041109: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-10 14:13:45.041165: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-10 14:13:45.041169: W tensorflow/compiler/tf

SoundStream(
  (encoder): Sequential(
    (0): CausalConv1d(
      (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
    )
    (1): Sequential(
      (0): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (1): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(3,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (2): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(9,))
   

In [2]:
import sys
import torch
from importlib import reload
import network_models.soundstream_lstm.LSTM_dataset as lds
import torch.utils.data as data_utils
#from network_models.soundstream_lstm.LSTM_dataset  import AudioEmotionTessDataset
reload(lds)


tess_dataset = lds.AudioEmotionTessDataset("../tess")

encoded_dataset = lds.AudioEmotionTessSoundStreamEncodedDataset(dataSet= tess_dataset, soundStream=soundstream)

In [3]:
import importlib
from pathlib import Path
import torch
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundstream_lstm.small_model as sm
importlib.reload(sm)


import network_models.soundstream_lstm.soundstream_datacollator as datacollator
importlib.reload(datacollator)
data_collator = datacollator.SoundStreamDataCollator(200)

# print(encoded_dataset.__getitem__(3).shape)
model = sm.EmotionClassifierSevenEmos().to("cuda")
tensor = torch.tensor(encoded_dataset.__getitem__(3)[0])
tensor = torch.nn.functional.pad(tensor, (0, 200 - tensor.shape[1], 0, 0))
tensor = tensor.flatten()
print(tensor[0].shape)


torch.Size([])


In [4]:
from torch.utils.data import Subset
import numpy as np
from network_models.soundstream_lstm.soundstream_datacollator import SoundstreamModelTrainer
from transformers import TrainingArguments, EvalPrediction

is_regression = False
from sklearn.model_selection import train_test_split

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    trainSet = Subset(dataset, train_idx)
    valSet = Subset(dataset, val_idx)
    return trainSet, valSet

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=0)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


train_dataset, eval_dataset = train_val_dataset(encoded_dataset, val_split=0.25)

training_args = TrainingArguments(
    output_dir="content/soundStreamClassModelsEmo",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=5.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)
trainer = SoundstreamModelTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

Using cuda_amp half precision backend


In [5]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#to override the collate_fn in dataloader because we have different sizes
def collate_fn(batch):
    return tuple(zip(*batch))

def pad_inputs(batch):
    input_batch = [torch.nn.functional.pad(torch.tensor(input_val).to("cuda"),
                        (0, 400 - input_val.shape[1], 0, 0))
                       for input_val in batch]
    return torch.stack((input_batch[0], input_batch[1]))

def pad_labels(batch):
    enc_list=[]
    for x in batch:
        a = np.zeros(7,)
        a[x] = 1
        enc_list.append(a)
    input_labels = [torch.tensor(label, dtype=torch.float32).to("cuda")
                    for label in enc_list]

    return torch.stack((input_labels[0], input_labels[1]))

In [7]:
import torch.optim as optim

learning_rate = 1e-4
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=10,
                                          shuffle=True, num_workers=2)
evalLoader = torch.utils.data.DataLoader(eval_dataset, batch_size=10,
                                          shuffle=True, num_workers=2)
train_dataset[1]

(array([[ 0.30637577, -0.3240556 , -0.32908133, ...,  0.        ,
          0.        ,  0.        ],
        [-0.40834558, -0.06076266, -0.00050882, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.08495785,  0.27482823, -0.03164213, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.32027078,  0.28586814,  0.36783856, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.02696363, -0.08964451, -0.18721043, ...,  0.        ,
          0.        ,  0.        ],
        [-0.07458303, -0.1834232 , -0.1924202 , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 array([4]))

In [32]:
from torch.utils.data import DataLoader
from torch import nn
from sklearn.preprocessing import OneHotEncoder# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

def encodeLabel(label):
    return enc.fit_transform(label).toarray()


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

learning_rate = 1e-3
batch_size = 1
epochs = 10


train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        # Compute prediction and loss
        pred = model(X)
        # print()
        #y = torch.tensor(encodeLabel(y.cpu())).to("cuda")
        y = torch.zeros(7, dtype=torch.long).to("cuda").scatter_(dim=0, index=y[0][0], value=1)
        loss = loss_fn(pred, y[0])

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to("cuda")
            y = y.to("cuda")
            pred = model(X.to("cuda"))
            test_loss += loss_fn(pred, y.to("cuda")).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.863275  [    0/ 2100]
loss: 1.246828  [  100/ 2100]
loss: 1.953896  [  200/ 2100]
loss: 1.854637  [  300/ 2100]
loss: 1.851477  [  400/ 2100]
loss: 1.848317  [  500/ 2100]
loss: 1.844934  [  600/ 2100]
loss: 2.002946  [  700/ 2100]
loss: 1.838379  [  800/ 2100]
loss: 1.835320  [  900/ 2100]
loss: 1.832255  [ 1000/ 2100]
loss: 1.828984  [ 1100/ 2100]
loss: 1.825358  [ 1200/ 2100]
loss: 1.821835  [ 1300/ 2100]
loss: 1.818582  [ 1400/ 2100]
loss: 1.814878  [ 1500/ 2100]
loss: 1.811035  [ 1600/ 2100]
loss: 1.807074  [ 1700/ 2100]
loss: 1.803721  [ 1800/ 2100]
loss: 1.963657  [ 1900/ 2100]
loss: 1.795836  [ 2000/ 2100]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [10]:
# for epoch in range(300):
#     running_loss = 0.0
#     for i, data in enumerate(trainLoader, 0):
#         #get input values and labels from the batch of the dataloaded
#         # print(data)
#         inputs, labels = data[0].to("cuda"), data[1].to("cuda")
#
#         # labels = data[1]
#         #print(labels)
#         #pad inputs
#         # padded_inputs = pad_inputs(inputs)
#         padded_labels = pad_labels(labels)
#         #print(padded_inputs)
#         #print(padded_labels)
#         # zero the parameter gradients
#         optimizer.zero_grad()
#
#         # forward + backward + optimize
#         outputs = model(inputs)
#         #print(outputs)
#         loss = criterion(outputs.to(torch.float32), labels.to(torch.float32))
#         #print(loss)
#         loss.backward()
#         optimizer.step()
#
#
#         # print statistics
#         running_loss += loss.item()
#
#         # valid_loss = 0.0
#         #
#         # for i, data in enumerate(evalLoader, 0):
#         #     inputs= data[0]
#         #     labels = data[1]
#         #     padded_inputs = pad_inputs(inputs)
#         #     padded_labels = pad_labels(labels)
#         #
#         #     outputs = model(padded_inputs)
#         #     loss = criterion(outputs, padded_labels)
#         #     valid_loss = loss.item() * padded_inputs.size(0)
#         #
#         # print(valid_loss)
#
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0
#
# print('Finished Training')

In [ ]:
# trainer.train()
